# TODO：概要

# ライブラリのインポート

In [ ]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

In [ ]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [ ]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# ユーティリティ関数

In [ ]:
def agent_run_outputs(thread_id, project_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = project_client.agents.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                project_client.agents.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

In [ ]:
# MCP ツールの定義
mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

# ツールセットの定義
toolset = ToolSet()
toolset.add(mcp_tool)


## エージェントの作成

In [ ]:
mcp_agent = project_client.agents.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="mcp_agent",
    instructions=(
        "あなたは計算や図表の出力に特化したアシスタントです。"
        "利用可能なツールを確認し、必要に応じて使用してください。"    
        "なお、グラフを描画する際は日本語が文字化けするため、ラベルやタイトルは英語で出力してください。"    
    ),
    description=(
        "Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。"
    ),
    tools=toolset.definitions,
)
print(f"Created Agent. AGENT_ID: {mcp_agent.id}")


Created Agent. AGENT_ID: asst_RbJ7n9txIVLA3eTyWuC3D0DV


In [ ]:
# カスタム関数の自動呼出しを有効化
project_client.agents.enable_auto_function_calls(toolset)

# スレッドの作成

In [ ]:
# Thread の作成
thread = project_client.agents.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_yN6593HGaVHOlThTJRaWlMtG


# ユーザーメッセージの追加

In [ ]:
# メッセージの追加
user_message = "Tokyoの今日の気温を教えて。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_RcVdh6YwnBjm2cb3xZyb2xEi


## Run の実行

In [ ]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=mcp_agent.id
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

USER: Tokyoの今日の気温を教えて。
ASSISTANT: 東京（Tokyo）の今日（2025年7月26日）の平均気温は28.9℃です。


※ ここで、Azure AI Foundry 上の Web UI からエージェントのトレースを確認してみよう。

## Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [ ]:
# 変数の定義
agent_env_key = "FOUNDRY_MCP_AGENT_ID"
agent_env_value = mcp_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')

# .envの自動読み込み（以降os.environで参照可能）
load_dotenv(env_path)
